In [9]:
import pandas as pd
import numpy as np
import re

In [10]:
df_minjust = pd.read_csv('data/Minjust2018.csv', 
                 header=None, 
                 usecols=[4, 27, 28, 33])
df_minjust.fillna('', inplace=True)
df_minjust.rename(columns={4: 'company_name', 
                           27: 'director', 
                           28: 'activity', 
                           33: 'founders'},
                  inplace = True)

In [11]:
df_decl = pd.read_csv('data/declaration.csv', 
                      header=None, 
                      usecols=[1, 2, 3, 4, 5])
df_decl.fillna('', inplace=True)
df_decl.rename(columns={1: 'surname', 
                        2: 'name', 
                        3: 'patronymic', 
                        4: 'workplace',
                        5: 'position'},
                  inplace = True)

In [12]:
def search_df(df, column, pattern):
    return df.loc[df[column].str.contains(pattern)]
    
def child_regex(surname, name):
    pattern = re.compile(surname +
                         r'а?\s+\w+\s+' + # First name
                         remove_vowels(name) + # Patronymic
                         r'|' + 
                         r'\w+\s+' + # First name
                         remove_vowels(name) + r'\w+\s+' + # Patronymic
                         surname,
                         re.IGNORECASE)
    return pattern

def remove_vowels(name):
    while True:
        if name[-1] not in ['а', 'о', 'и', 'е', 'ё', 'э', 'ы', 'у', 'ю', 'я', 'й']:
            break
        else:
            name = name[: -1]
    return name

def brother_regex(patronymic, surname):
    pattern = re.compile(surname +
                         r'а?\s+\w+\s+' + # First name
                         patronymic + # Patronymic
                         r'|' + 
                         r'\w+\s+' + # First name
                         patronymic + # Patronymic
                         surname,
                         re.IGNORECASE)
    return pattern

In [13]:
df_decl.loc[923: 993]

,surname,name,patronymic,workplace,position
923,Нарынбекова,Айсалкын,Омуралиевна,Конституционная палата ВС КР,Судья
924,Айдарбекова,Чинара,Аскарбековна,Конституционная палата ВС КР,Судья-секретарь
925,Бобукеева,Меергуль,Рыскулбековна,Конституционная палата ВС КР,Судья
926,Мамыров,Эркинбек,Тобокелович,Конституционная палата ВС КР,Заместитель председателя
927,Жусупбеков,Нурбек,Исраилович,Кочкорская районная государственная администра...,Глава
928,Садыкова,Элмира,Аманатовна,Кочкорская районная государственная администра...,Заместитель главы
929,Керимкулов,Нурлан,Бейшенбекович,Кочкорская районная государственная администра...,Первый заместитель главы
930,Асанов,Кайратбек,Муратбекович,Кочкорская районная государственная администра...,Руководитель аппарата
931,Маткалыков,Асилбек,Жоробаевич,Кызыл-Кийский городской кенеш,Председатель
932,Аширов,Жанышбек,Сайдилаевич,Кызыл-Кийский городской суд,Председатель


In [14]:
columns = ['company_name', 'director', 'activity', 'founders', 'surname', 'name', 'patronymic', 'workplace', 'position']
result = pd.DataFrame(columns=columns)

In [15]:
for index, row in df_decl.loc[781: 851].iterrows():
    if row['patronymic'] != '':
        search = search_df(df_minjust, 'founders', brother_regex(row['patronymic'], row['surname'])).copy()
        for column in df_decl.columns:
            search[column] = row[column]
        result = result.append(search, ignore_index=True)


In [16]:
result

,company_name,director,activity,founders,surname,name,patronymic,workplace,position
0,"Общество с ограниченной ответственностью ""ЮрДеМ""",Галиев Мелис Душаевич,Деятельность в области юстиции и правосудия,"Галиев Мелис Душаевич, Ибраимжанов Адилетбек С...",Ибраимжанов,Болотбек,Сапарбаевич,Жогорку Кенеш Кыргызской Республики,экс-депутат
1,"Общество с ограниченной ответственнотью ""Алай ...",Кадамбаев Мухтарбек Токтомунович,Деятельность общественных организаций (объедин...,Ибраимжанов Болотбек Сапарбаевич,Ибраимжанов,Болотбек,Сапарбаевич,Жогорку Кенеш Кыргызской Республики,экс-депутат
2,"ОсОО ""Ошская фабрика нетканных материалов"" (АО...",Атамбаев Керим,,"Атамбаев Керим Жанакович, Ибраимжанов Болотбек...",Ибраимжанов,Болотбек,Сапарбаевич,Жогорку Кенеш Кыргызской Республики,экс-депутат
3,"Общество с ограниченной ответственностью ""Атла...",Мамурасулова Элмира Тургунбаевна,Добыча отделочного и строительного камня,"Ибраимжанов Адилетбек Сапарбаевич, Мамурасулов...",Ибраимжанов,Болотбек,Сапарбаевич,Жогорку Кенеш Кыргызской Республики,экс-депутат
4,Политическая партия «Республиканская партия С...,Джекшенкулов Аликбек Джекшенкулович,Деятельность политических организаций,"Джекшенкулов Аликбек Джекшенкулович, Аргымбаев...",Жылкыбаев,Узарбек,Казиевич,Жогорку Кенеш Кыргызской Республики,Депутат
5,"Общественное объединение ""Алга Бостери""",Жылкыбаев Узарбек Казиевич,"Деятельность прочих общественных организаций, ...","Жылкыбаев Узарбек Казиевич, Атаев Акжол Сабарб...",Жылкыбаев,Узарбек,Казиевич,Жогорку Кенеш Кыргызской Республики,Депутат
6,Общество с ограниченной ответственностью «ЭКИД...,Жылкыбаев Узарбек Казиевич,63.30.0 - Туристическая и экскурсионная деятел...,"Куренбеков Тимур Амантаевич, Жылкыбаев Узарбек...",Жылкыбаев,Узарбек,Казиевич,Жогорку Кенеш Кыргызской Республики,Депутат
7,"""Общественный фонд Жусупа Абдрахманова первого...",Усебаев Мирбек Сагынович,"Деятельность прочих общественных организаций, ...","Исаматов Казыбек Барыктабасович, Маматалиев Ба...",Жылкыбаев,Узарбек,Казиевич,Жогорку Кенеш Кыргызской Республики,Депутат
8,"Общество с ограниченной ответственностью ""ИСМ...",Матраимов Исламбек Исмаилович,Деятельность агентов по оптовой торговле товар...,"Матраимов Исламбек Исмаилович, Матраимов Русла...",Матраимов,Искендер,Исмаилович,Жогорку Кенеш Кыргызской Республики,Депутат
9,"Общественный фонд ""Правовая Клиника ""Адилет""",Джакупова Чолпон Идиновна,"Деятельность прочих общественных организаций, ...",Джакупова Чолпон Идиновна,Джакупова,Чолпон,Идиновна,Жогорку Кенеш Кыргызской Республики,Депутат


In [18]:
result.to_pickle('Suban_923_993_Brothers.pkl')